<a href="https://colab.research.google.com/github/chopstickexe/deep-learning-from-scratch-2/blob/master/ch07_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下準備

$$
\newcommand{\vect}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
$$

## 数式の表記

変数: 小文字イタリック $x$

定数: 大文字イタリック $X$

ベクトル: 小文字ローマン体太字 $\vect{x}$

行列: 大文字ローマン体太字 $\mat{X}$

## 公式実装のclone

In [1]:
!git clone --depth=1 https://github.com/oreilly-japan/deep-learning-from-scratch-2.git
import sys 
sys.path.append('deep-learning-from-scratch-2')

fatal: destination path 'deep-learning-from-scratch-2' already exists and is not an empty directory.


# 7章 RNNによる文章生成

6章で実装した言語モデルが出力する確率分布に従って単語をサンプリングし，文章を生成する．

## 7.1 言語モデルを使った文章生成（`RnnlmGen`，`BetterRnnlmGen`クラス）

In [2]:
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
  def generate(self, start_id, skip_ids=None, sample_size=100): 
    '''
    start_id: 開始単語のID
    skip_ids: <unk>など生成に使いたくない単語のリスト
    sample_size=文長
    '''
    word_ids = [start_id]

    x = start_id
    while len(word_ids) < sample_size:
      x = np.array(x).reshape(1, 1)
      score = self.predict(x)  # Softmaxレイヤの直前の出力
      p = softmax(score.flatten())  # flatten(): ndarrayを一次元の配列化する

      sampled = np.random.choice(len(p), size=1, p=p)  # サンプリング．len(p)は生成されるサンプルの値域最大値，sizeは生成されるサンプル数，pはサンプルする際に使われる確率分布

      if (skip_ids is None) or (sampled not in skip_ids):
        x = sampled
        word_ids.append(int(x))

    return word_ids

PTBデータセットで学習した普通のLSTMモデルを利用して文章生成．

In [3]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('deep-learning-from-scratch-2/ch06/Rnnlm.pkl')

# start文字とskip文字の設定
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文章生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you mean which was in an official choice.
 sight relief can rise around in the year.
 other players could suppliers some other of the film drawn and can decide why the country had n't bolster the going have been grabbed good worse.
 for the first moment hepatitis a leadership and long president should raise questions close to areas nuclear radio while give both the coleman and a small sudden called him.
 china 's nations operating for the industry might also reduce law available and considers significant debt owned technologies which can home the economy.
 that


LSTMレイヤを2層にして，各層にDropoutを入れ，さらにEmbeddingの転置行列（H x V, Hは入力単語を表現する隠れベクトルのサイズ，Vは元の語彙数）をAffine Layerの重みにそのまま使うモデル（`ch06/better_rnnlm.py`）に変更すると，生成される文章の質が上がる．
（より言語のルールを守った文章になる）

In [4]:
!curl -o deep-learning-from-scratch-2/ch06/BetterRnnlm.pkl https://www.oreilly.co.jp/pub/9784873118369/BetterRnnlm.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37.7M  100 37.7M    0     0  11.1M      0  0:00:03  0:00:03 --:--:-- 11.1M


In [5]:
from ch06.better_rnnlm import BetterRnnlm  # LSTMレイヤを2層利用して、各層でDropoutするモデル
from ch07.rnnlm_gen import BetterRnnlmGen  # RnnlmGenの継承クラスをRnnlmからBetterRnnlmに変更しただけのクラス

model = BetterRnnlmGen()
model.load_params('deep-learning-from-scratch-2/ch06/BetterRnnlm.pkl')

# start文字とskip文字の設定
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文章生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you supposedly reflect the ability of the new york stock exchange of manhattan.
 the mega-issues in a company that provides data any embarrassing priority which is n't have one major debt the week does n't be proven during the past two sessions.
 it did n't cover the letters of a long-term investment sales.
 in other financial markets markets.
 stock futures ended slightly higher in thin trading because the stock-market rise price is only handful ahead.
 stock prices closed lower in thin trading.
 the board cited one of the market 's recent sell-off in gold
